In [1]:
from IPython.core.display import Markdown
from dotenv import load_dotenv
load_dotenv(dotenv_path="../../../.env", override=True)

import os
from anthropic import Anthropic
from langsmith import traceable
from typing import List
import nest_asyncio
from utils import get_vector_db_retriever

USER_AGENT environment variable not set, consider setting it to identify your requests.
/Users/manan/PycharmProjects/langsmith-course/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
%load_ext autoreload
%autoreload 2

In [7]:
MODEL_PROVIDER = "claude"
MODEL_NAME = "claude-3-5-haiku-latest"
APP_VERSION = 1.0
RAG_SYSTEM_PROMPT = """You are an assistant for making cold emails for students that go to professors.
Use the following pieces of retrieved context to generate emails to be sent to the professor.
If you don't know what to write in the mail, just say that you don't know.
"""

anthropic_client = Anthropic(api_key=os.environ.get("ANTHROPIC_API_KEY"))
nest_asyncio.apply()
retriever = get_vector_db_retriever()

@traceable
def retrieve_documents(question: str):
    return retriever.invoke(question)   # NOTE: This is a LangChain vector db retriever, so this .invoke() call will be traced automatically

@traceable
def generate_response(question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    messages = [
        {
            "role": "user",
            "content": f"Context: {formatted_docs} \n\n Question: {question}"
        }
    ]
    return call_claude(messages)

@traceable
def call_claude(
    messages: List[dict], model: str = MODEL_NAME, temperature: float = 0.0
) -> str:
    return anthropic_client.messages.create(
        model=model,
        messages=messages,
        system=RAG_SYSTEM_PROMPT,
        max_tokens=1024,
        temperature=temperature,
    ).content[0].text

@traceable(metadata={"Model": "all-MiniLM-l6-v2"})
def langsmith_rag(question: str):
    documents = retrieve_documents(question)
    response = generate_response(question, documents)
    return response

In [10]:
ans = langsmith_rag("interest in Machine learning and Natural language processing", langsmith_extra={"metadata": {"markdown": "True"}})

In [11]:
from IPython.display import Markdown

In [12]:
Markdown(ans)

Based on the context provided, I would recommend drafting an email to Nazil Perveen, who is an Assistant Professor in the Department of Computer Science and Engineering. Machine Learning and Natural Language Processing are typically computer science domains, so she might be the most relevant contact.

Here's a draft email:

Subject: Inquiry about Machine Learning and NLP Research Opportunities

Dear Dr. Nazil Perveen,

I hope this email finds you well. I am a student interested in Machine Learning and Natural Language Processing, and I am reaching out to explore potential research opportunities or guidance in these fields within the Department of Computer Science and Engineering.

I am particularly keen to understand:
- Current research projects in ML and NLP
- Potential undergraduate/graduate research opportunities
- Recommended courses or resources in these domains

Would you be available for a brief discussion about these areas of study? I would greatly appreciate any insights you could share.

Thank you for your time and consideration.

Sincerely,
[Your Name]

Note: This email template is generic and should be personalized based on your specific background, interests, and any prior research about the professor's work.